In [13]:
import tkinter
from tkinter import filedialog
from tkinter import *
import pandas as pd
import json
import geocoder  
import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure
import numpy as np
import pymysql as mariadb
import pycountry

import matplotlib
matplotlib.use('TkAgg')

import tkinter as tk
from tkinter import ttk

from numpy import arange, sin, pi
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
# implement the default mpl key bindings
#from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure

class UI(Frame):
    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.grid()
        self.master.title("Intelligence Chart")
        self.master.geometry("680x400")
        self.select_col = []
        self.select_row = []
        self.show()
        
    def show(self):
        #self.frm = Frame(self.master)
        #self.frm.grid(row=1, column=0, sticky=N+S)
        self.master.rowconfigure(1, weight=1)
        self.master.columnconfigure(1, weight=1)
        
           
        self.open_text = ttk.Label(self,text="Open file")
        self.open_text.grid(row=0,column=0)
        self.button_open = ttk.Button(self, text="Excel", command=self.open_file)
        self.button_open.grid(row=0,column=1)
    
    def open_file(self):
        master.filename =  filedialog.askopenfilename(initialdir = "C:/Users/teer/Work_BuildingII",title = "Select file",filetypes = (('excel files', '*.xls *.xlsx *.xlsm'),("All files", "*.*")))
        self.filename = master.filename
        if self.filename != "" :
            self.convert_file()

    def convert_file(self):
        self.open_text.grid_remove() # remove widget
        self.button_open.grid_remove()
        self.df = pd.read_excel(open(self.filename,'rb'))
        col_name = list(self.df)
        col_type = self.df.dtypes
        self.Dimensions = []
        self.Measures = []
        # check column type 
        for i in range(len(col_type)):
            # if column is numeric type
            if(str(col_type[i]) == 'object'):
                self.Dimensions.append(col_name[i])
            elif(str(col_type[i]) == 'datetime64[ns]'):
                #print(col_type[i])
                self.Dimensions.append(col_name[i])
                
            else :
                #print(col_name[i])
                unique_elements = np.unique(pd.DataFrame(self.df, columns = [col_name[i]]))
                #print(len(unique_elements))
                #print(len(df.index))
                if len(unique_elements) == len(self.df.index) :
                    self.Dimensions.append(col_name[i])
                else :
                    self.Measures.append(col_name[i])
                    
        self.Dimensions.sort()
        self.Measures.sort()
        self.Dimensions.append("Measure Names")
        self.Measures.extend(["Number of Records","Measure Values"])
        self.show_dimen_mea()
        
    def show_dimen_mea(self):
        
        self.text_dimen = ttk.Label(self,text="Dimensions")
        self.text_dimen.grid(row=1,column=0,sticky=E+W)
        self.text_mea = ttk.Label(self,text="Measures")
        self.text_mea.grid(row=5,column=0,sticky=E+W)
        
        self.list_dimention = tkinter.Listbox(self,height = 1)
        self.list_measure = tkinter.Listbox(self,height = 1)
        self.list_dimention.grid(row = 2, column = 0, sticky="nsew")
        self.list_measure.grid(row =6,column = 0, sticky="nsew")
        self.change_to_mea = ttk.Button(self, text="change to measure", command=self.change_mea)
        self.change_to_mea.grid(row=4,column=0)
        self.change_to_dimen = ttk.Button(self, text="change to dimension", command=self.change_dimen)
        self.change_to_dimen.grid(row=8,column=0)
        
        self.scroll_diy = tkinter.Scrollbar(self,orient=tkinter.VERTICAL,command=self.list_dimention.yview)
        self.scroll_diy.grid(row=2, column=1, sticky=tkinter.NW+tkinter.S)
        self.scroll_dix = tkinter.Scrollbar(self,orient=tkinter.HORIZONTAL,command=self.list_dimention.xview)
        self.scroll_dix.grid(row=3, column=0, sticky='news')
        self.list_dimention.config(yscrollcommand=self.scroll_diy.set,xscrollcommand=self.scroll_dix.set)
        #self.scroll_diy.config()
        #self.scroll_dix.config()
        
        self.scroll_meay = tkinter.Scrollbar(self,orient=tkinter.VERTICAL,command=self.list_measure.yview)
        self.scroll_meay.grid(row=6, column=1, sticky=tkinter.NW+tkinter.S)
        self.scroll_meax = tkinter.Scrollbar(self,orient=tkinter.HORIZONTAL,command=self.list_measure.xview)
        self.scroll_meax.grid(row=7, column=0, sticky='news')
        self.list_measure.config(yscrollcommand=self.scroll_meay.set,xscrollcommand=self.scroll_meax.set)
        #self.scroll_meay.config()
        #self.scroll_meax.config()
        
        #self.list_dimention.bind('<<ListboxSelect>>', self.onselect_dimen)
        #self.list_measure.bind('<<ListboxSelect>>', self.onselect_mea)
        
        for i in self.Dimensions:
            self.list_dimention.insert("end",i)
        for i in self.Measures:
            self.list_measure.insert("end",i)
        # show selection row and colum.
        self.select_row_col()
        
    def change_mea(self):
        get = self.list_dimention.curselection()
        if get :
            if self.Dimensions[get[0]] in self.Dimensions:
                self.Dimensions = np.delete(self.Dimensions,get[0], 0)
            keep = self.list_dimention.get(get[0])
            self.Measures = np.append(self.Measures,[keep])
            self.list_measure.delete(0,"end")
            for i in self.Measures:
                self.list_measure.insert("end",i)
            # update dimension.
            self.list_dimention.delete(0,"end")
            for i in self.Dimensions:
                self.list_dimention.insert("end",i)
            self.list_dimention.selection_clear(0, END)
            
    
    def change_dimen(self):
        get = self.list_measure.curselection()
        if get :
        # delete value in measure.
            if self.Measures[get[0]] in self.Measures:
                self.Measures = np.delete(self.Measures,get[0], 0)
            keep = self.list_measure.get(get[0])
            print(self.Dimensions)
            self.Dimensions = np.append(self.Dimensions,[keep])
            print(self.Dimensions)
            # update dimension.
            self.list_dimention.delete(0,"end")
            for i in self.Dimensions:
                self.list_dimention.insert("end",i)
            # update measure.
            self.list_measure.delete(0,"end")
            for i in self.Measures:
                self.list_measure.insert("end",i)
            self.list_measure.selection_clear(0, END)
        
    
    def select_row_col(self):
        
        self.col_text = ttk.Label(self,text="Column :")
        self.col_text.grid(row=0,column=3)
        self.add_col = ttk.Button(self, text="Add", command=self.add_col)
        self.add_col.grid(row=0,column=1)
        self.del_col = ttk.Button(self, text="Delete", command=self.del_col)
        self.del_col.grid(row=0,column=2)
        self.list_col = tkinter.Listbox(self,height = 1)
        self.list_col.grid(row = 0, column = 4, sticky="nsew")
        self.scroll_col = tkinter.Scrollbar(self,orient=tkinter.VERTICAL)
        self.scroll_col.grid(row=0, column=5, sticky=tkinter.NW+tkinter.S)
        self.list_col.config(yscrollcommand=self.scroll_col.set)
        self.scroll_col.config(command=self.list_col.yview)
        ## row
        self.row_text = ttk.Label(self,text="Row :")
        self.row_text.grid(row=0,column=8)
        self.add_row = ttk.Button(self, text="Add", command=self.add_row)
        self.add_row.grid(row=0,column=6)
        self.del_row = ttk.Button(self, text="Delete", command=self.del_row)
        self.del_row.grid(row=0,column=7)
        self.list_row = tkinter.Listbox(self,height = 1)
        self.list_row.grid(row = 0, column = 9, sticky="nsew")
        self.scroll_row = tkinter.Scrollbar(self,orient=tkinter.VERTICAL)
        self.scroll_row.grid(row=0, column=10, sticky=tkinter.NW+tkinter.S)
        self.list_row.config(yscrollcommand=self.scroll_row.set)
        self.scroll_row.config(command=self.list_row.yview)
        
    def add_col(self):
        dimen = self.list_dimention.curselection()
        mea = self.list_measure.curselection()
        
        if dimen :
            get = self.list_dimention.get(dimen[0])
            if get not in self.select_col:
                self.select_col = np.append(self.select_col,[get])
                self.show_listcol()
            self.list_dimention.selection_clear(0, END)
        elif mea :
            get = self.list_measure.get(mea[0])
            if get not in self.select_col:
                self.select_col = np.append(self.select_col,[get])
                self.show_listcol()
            self.list_measure.selection_clear(0, END)
        
    
    def show_listcol(self):
        self.list_col.delete(0,"end")
        for i in self.select_col :
            self.list_col.insert("end",i)
    
    def del_col(self):
        get = self.list_col.curselection()
        if get :
            if self.select_col[get[0]] in self.select_col:
                self.select_col = np.delete(self.select_col,get[0], 0)
            self.show_listcol()
            self.list_col.selection_clear(0, END)
        
    def add_row(self):
        dimen = self.list_dimention.curselection()
        mea = self.list_measure.curselection()
        if dimen :
            get = self.list_dimention.get(dimen[0])
            if get not in self.select_row:
                self.select_row = np.append(self.select_row,[get])
                self.show_listrow()
            self.list_dimention.selection_clear(0, END)
        elif mea :
            get = self.list_measure.get(mea[0])
            if get not in self.select_row:
                self.select_row = np.append(self.select_row,[get])
                self.show_listrow()
            self.list_measure.selection_clear(0, END)
    
    def show_listrow(self):
        self.list_row.delete(0,"end")
        for i in self.select_row :
            self.list_row.insert("end",i)
    
    def del_row(self):
        get = self.list_row.curselection()
        if get :
            if self.select_row[get[0]] in self.select_row:
                self.select_row = np.delete(self.select_row,get[0], 0)
            self.show_listrow()
            self.list_row.selection_clear(0, END)
            
    def chart(self,d):
        print("chart!!")
        
        f = Figure(figsize=(5, 4), dpi=100)
        a = f.add_subplot(111)
        collabel = []
        collabel.extend((self.select_dimen, self.select_mea))
        dat = d.loc[:5, collabel]
        dat = dat.applymap(str)
        dat = dat.values
        
        print(dat)
        print(collabel)
        n_rows = len(dat)
        index = np.arange(len(collabel)) + 0.3
        bar_width = 0.4

# Initialize the vertical-offset for the stacked bar chart.
        y_offset = np.zeros(len(collabel))
        #collabel=("col 1", "col 2")
        for row in range(n_rows):
            a.bar(index, dat[row], bar_width, bottom=y_offset)
            y_offset = y_offset + dat[row]
            cell_text.append(['%1.1f' % (x / 1000.0) for x in y_offset])

        a.table(cellText=dat,colLabels=collabel,loc='bottom')
        #a.hist(dat, bins =5)

        fr = tkinter.Frame(self)
        fr.grid(row=1, column=2, columnspan=4)

        canvas = FigureCanvasTkAgg(f, master=fr)
        canvas.show()
        canvas.get_tk_widget().pack()

        toolbar = NavigationToolbar2TkAgg(canvas, fr)
        toolbar.update()
        canvas._tkcanvas.pack()
        
        
    def get(self):
        if(np.where( self.select_dimen == "Measure Names" ) != ""):
            #print(self.df[self.select_mea])
            measure_name = np.sum(list(self.df[self.select_mea]),axis=0)
            print(measure_name)
        elif(np.where( self.select_mea == "Number of Records") != ""):
            unique_elements, counts_elements = np.unique(list(self.df[self.select_dimen]), return_counts=True)
            num_record = np.concatenate(([unique_elements], [counts_elements]), axis=0).T
            print(num_record)
        elif(np.where( self.select_mea == "Measure Values") != ""):
            measure_value = np.sum(list(self.df[self.select_mea]),axis=0)
            print(measure_value)
        
        
class Intel_chart(Frame): 
    
    def __init__(self, master=None):
        Frame.__init__(self, master)
            
                    
    def check_database(self):
        
        encode = self.md5sum(self.filename) # check encode read
        # defind database
        mariadb_connection = mariadb.connect(user="root", password="0897737223", database='chart') 
        cursor = mariadb_connection.cursor()
        sql = "SELECT id FROM file where code ='%s' " % (encode)
        # Execute the SQL command
        cursor.execute(sql)
        no = cursor.fetchall()
        check = False
        if len(no) != 0 :
            check = True
        else:
            sql = "SELECT COUNT(*) FROM file"
            cursor.execute(sql)
            no = cursor.fetchall()
            no = no[0][0] + 1
            
        if check == False :
            sql = "INSERT INTO file(id, code) VALUES(%s, %s)"
            data_insert = (no,encode)
            # Execute the SQL command
            cursor.execute(sql,data_insert)
            # Commit your changes in the database
            mariadb_connection.commit()
       
            
master = Tk()
app = UI(master)
app.mainloop()


